In [1]:
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger

In [32]:
# Base User class
class User:
    def __init__(self, name, broker, **kwargs):

        # The User class initializes with name, broker, and other attributes that vary depending on the broker.

        self.name = name
        self.broker = broker

        # Set dynamic attributes based on the broker
        self.username = kwargs.get("username", None)
        self.api_key = kwargs.get("api_key", None)
        self.pwd = kwargs.get("pwd", None)
        self.totp_token = kwargs.get("totp_token", None)
        self.api_obj = None  # Placeholder for api obj
        self.auth_token = None  # Placeholder for login access token
        self.refresh_token = None  # Placeholder for login refresh token
        self.feed_token = None  # Placeholder for login feed token

        # Call the login method automatically when the object is created
        print(self.user_login())

    def get_user_info(self):

        # Returns the user information in a formatted string.
        return f"Name: {self.name}, Broker: {self.broker}, Username: {self.username},"


    def user_login(self):

        # Simulates user login and generates an access token.

        print(f"Logging in {self.username} to {self.broker}...")

        if self.broker == "AngelOne":
            return self._login_angel_one()
        elif self.broker == "OtherBroker":
            return self._login_other_broker()
        else:
            return "Unsupported broker. Please provide valid credentials."

    def _login_angel_one(self):

        # Login using Angel One's SmartAPI.

        try:
            token = self.totp_token
            totp = pyotp.TOTP(token).now()
        except Exception as e:
            logger.error("Invalid Token: The provided token is not valid.")
            raise e

        try:
            # SmartAPI login for Angel One
            self.api_obj = SmartConnect(api_key=self.api_key)

            data = self.api_obj.generateSession(self.username, self.pwd, totp)

            self.auth_token = data['data']['jwtToken']          # Store access token
            self.refresh_token = data['data']['refreshToken']   # Store refresh_token
            self.feed_token = self.api_obj.getfeedToken()       # Store Feed token

            return f"Login {self.api_obj.getProfile(self.refresh_token)["message"]}"

        except Exception as e:
            return f"Angel One login failed: {str(e)}"

    def _login_other_broker(self):
        """
        Placeholder login function for other brokers.
        """

    def place_order(self, order_details):
        """
        Simulates placing an order.
        """



# Parent User class with broker-specific attributes and children management
class ParentUser(User):
    def __init__(self, name, broker, **kwargs):

        # ParentUser class inherits from User and can manage child users.

        super().__init__(name, broker, **kwargs)
        self.children = []

    def add_child(self, child):

        # Adds a ChildUser to the ParentUser's list of children.

        self.children.append(child)

    def list_children(self):

        # Returns a list of the usernames of all children under the ParentUser.

        return [child.name for child in self.children]

    def place_order_for_child(self, order_details):
         # Places an order for all children.
        for child in self.children:
            child_order_result = child.place_order(order_details)
            print(f"Child Order: {child_order_result}")

    def place_order_for_all(self, order_details):

        # Places an order for both the parent and all children.

        parent_order_result = self.place_order(order_details)
        print(parent_order_result)
        self.place_order_for_child(order_details)



# Child User class (must be linked to a ParentUser)
class ChildUser(User):
    def __init__(self, name, broker, parent, **kwargs):

        # ChildUser class inherits from User and is linked to a ParentUser.

        super().__init__(name, broker, **kwargs)
        self.parent = parent
        parent.add_child(self)  # Automatically add the child to the parent's list

    def get_parent_info(self):

        # Returns the parent user's information.

        return f"Parent Name: {self.parent.name}, Parent Broker: {self.parent.broker}"



In [29]:
# Example Usage
# Creating a ParentUser with custom attributes based on the broker
parent = ParentUser(name="Yash",
                    broker="AngelOne",
                    api_key = 'bE7ipU6q',
                    username = 'Y58983127',
                    pwd = '1416',
                    totp_token = "NQST6OLTU6MP3PKRH2BLXUTGDI")

Logging in Y58983127 to AngelOne...


[I 241017 04:04:08 smartConnect:121] in pool


Login SUCCESS


In [30]:
# Creating ChildUser instances and linking them to the parent
child1 = ChildUser( name="Ravi",
                    broker="AngelOne",
                    parent=parent,
                    api_key = 'eskMiA2H',
                    username = 'R947822',
                    pwd = '1234',
                    totp_token = "E4FCOCHUNVRDG7ANFXK6D3OQVA")

Logging in R947822 to AngelOne...


[I 241017 04:04:13 smartConnect:121] in pool


Login SUCCESS


In [31]:
# Print user info
print(child1.get_parent_info())       # Parent user information
print(parent.list_children())         # Children user information

Parent Name: Yash, Parent Broker: AngelOne
['R947822']


In [34]:
# terminating Session
parent.api_obj.terminateSession('Y58983127')
child1.api_obj.terminateSession('R947822')

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': 'Logout Successfully'}

In [ ]:
# Placing an order for both parent and children
parent.place_order_for_all(order_details={"symbol": "AAPL", "qty": 10, "action": "buy"})